In [1]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.insert(0, '../JKBio/')
import epigenetics.ChIP_helper as chiphelper
import Helper as helper
import igv
import numpy as np
import pyBigWig
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
! gcsfuse --only-dir Chip_AML amlproject data/seqs

Using mount point: /home/jeremie/AMLproject/data/seqs
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [30]:
!fusermount -u data/seqs

In [34]:
! ls data/seqs/GEO

ls: cannot access 'data/seqs/GEO': No such file or directory


## finding all the relevant files for chipseq processing

In [19]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip

In [11]:
a

['gs://jkobject/Chip_AML/Chip/20180709_1_MP5773_S1.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20180709_3_MP5773_S3.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20180709_5_MP5773_S5.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20180709_6_MP5773_S6.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-11_MP7213-2of2_S4_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-13_MP7213-2of2_S5_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-14_MP7213-2of2_S6_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-2_MP7213-1of2_S2_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-4_MP7213-1of2_S4_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-5_MP7213-1of2_S5_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-6_MP7213-1of2_S6_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190718_MP-8_MP7213-2of2_S1_R1_001.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190826_10_MB7326-2of2_S5.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/20190826_11_MB7326-2of2_S6.fastq.gz

In [28]:
rename1 = {'gs://amlproject/Chip_AML/Chip/20180709_1_MP5773_S1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp631.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_3_MP5773_S3.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp633.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_5_MP5773_S5.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp635.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20180709_6_MP5773_S6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp636.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-11_MP7213-2of2_S4_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp748.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-13_MP7213-2of2_S5_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp750.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-14_MP7213-2of2_S6_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp751.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-2_MP7213-1of2_S2_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp739.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-4_MP7213-1of2_S4_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp741.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-5_MP7213-1of2_S5_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp742.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-6_MP7213-1of2_S6_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp743.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190718_MP-8_MP7213-2of2_S1_R1_001.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp745.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_10_MB7326-2of2_S5.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp761.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_11_MB7326-2of2_S6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp762.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/20190826_9_MB7326-2of2_S4.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp760.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D0-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp8.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp14.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9_CEBPA_m.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp27.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/D9_INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp35.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP29.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp4.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP30.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp5.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/MP31.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp6.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2-INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp24.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2_CEBPA_m.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp41.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/OCI_AML2_INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp33.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_10-D8-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp156.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_12-MV411-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp115.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_15-MV411-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp118.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_18-D8-MED1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp160.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_19-M6-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp122.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_20-M6-RUNX1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp123.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_21-M6-MYB.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp124.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_22-M6-ETV6.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp125.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_24-M6-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp171.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_25-M6-SP1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp127.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_26-M6-MED1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp128.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_27-MV411-RUNX1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp129.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_27-OCIAML2-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp168.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_31.M6.INPUT.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp98.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_41-M6-IGG.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp143.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_6-M6-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp109.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_7-D8-CEBPA.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp153.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_8-D8-SPI1.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp154.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/Sample_9-D8-MYB.fastq.gz':'gs://amlproject/Chip_AML/Chip/mp155.fastq.gz'
}

## looking for duplicates

In [20]:
import re
mps=[]
for i in a:
    res = re.search('mp(\d+)', i)
    if res is not None:
        mps.append(res.group(0))

In [ ]:
mps.sort()

In [23]:
mps

['mp106',
 'mp112',
 'mp116',
 'mp117',
 'mp134',
 'mp136',
 'mp157',
 'mp183',
 'mp183',
 'mp186',
 'mp187',
 'mp188',
 'mp193',
 'mp194',
 'mp195',
 'mp196',
 'mp197',
 'mp2',
 'mp203',
 'mp204',
 'mp205',
 'mp206',
 'mp207',
 'mp213',
 'mp214',
 'mp216',
 'mp217',
 'mp218',
 'mp230',
 'mp231',
 'mp232',
 'mp233',
 'mp299',
 'mp299',
 'mp300',
 'mp300',
 'mp301',
 'mp301',
 'mp302',
 'mp302',
 'mp304',
 'mp304',
 'mp305',
 'mp305',
 'mp308',
 'mp308',
 'mp316',
 'mp319',
 'mp319',
 'mp320',
 'mp320',
 'mp321',
 'mp321',
 'mp324',
 'mp324',
 'mp325',
 'mp325',
 'mp351',
 'mp352',
 'mp353',
 'mp354',
 'mp355',
 'mp356',
 'mp357',
 'mp358',
 'mp359',
 'mp424',
 'mp425',
 'mp426',
 'mp427',
 'mp428',
 'mp429',
 'mp430',
 'mp431',
 'mp433',
 'mp435',
 'mp45',
 'mp48',
 'mp50',
 'mp53',
 'mp56',
 'mp568',
 'mp569',
 'mp570',
 'mp575',
 'mp577',
 'mp58',
 'mp581',
 'mp582',
 'mp582',
 'mp583',
 'mp584',
 'mp585',
 'mp587',
 'mp587',
 'mp589',
 'mp590',
 'mp591',
 'mp592',
 'mp593',
 'mp594'

In [24]:
duplicates=[ 'gs://amlproject/Chip_AML/Chip/Sample_mp582-MV411-MEIS1-r1.fastq.gz',
'gs://amlproject/Chip_AML/Chip/Sample_mp587-MV411-IKZF1-r1.fastq.gz',
,
'gs://amlproject/Chip_AML/Chip/mp88-MV411-H3K27ac-r2.fastq.gz']

In [25]:
for val in duplicates:
    os.system('gsutil rm '+val)

In [26]:
paired_end_tomove= ['gs://amlproject/Chip_AML/Chip/mp304-D9-PU1-r2_S1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp305-SKNO1-PU1-r1_S2_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp320-MV411-ZEB2-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp319-MV411-FLI1-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp308-SKNO1-RUNX1-RUNX1T1-r1_S5_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp321-MV411-GFI1-r4_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp324-MV411-MEF2D-r1_R2.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp299-MV411-CEBPA-r3_S7_R2.fastq.gz'
 'gs://amlproject/Chip_AML/Chip/mp325-MV411-SP1-r2_R2.fastq.gz']

In [36]:
for val in paired_end_tomove:
    os.system('gsutil mv '+val+' gs://amlproject/Chip_AML/Chip/paired_end/')

## renamings

In [29]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [33]:
## IT SHOULD NOT CAUSE ANY PROBLEM TO CP R2 SINCE IT DOES NOT EXIST 
#(will throw an error and skip)
import re
import time
mps=[]
for i in a:
    res = re.search('mp(\d+)', i)
    if res is not None:
        print('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')
        time.sleep(2)
        os.system('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')

gsutil mv gs://jkobject/Chip_AML/Chip/Sample_25-mp568-MV411-ZEB2-r3.fastq.gz gs://jkobject/Chip_AML/Chip/mp568.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp230-SKNO1-MYB-r1_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp230.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp231-SKNO1-CEBPA-r1_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp231.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp232-SKNO1-ETV6-r1_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp232.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp233-SKNO1-RUNX1-r1_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp233.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp316-SKNO1-INPUT-r1.fastq.gz gs://jkobject/Chip_AML/Chip/mp316.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp582-MV411-MEIS1-r1.fastq.gz gs://jkobject/Chip_AML/Chip/mp582.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/Sample_mp587-MV411-IKZF1-r1.fastq.gz gs://jkobject/Chip_AML/Chip/mp587.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp106-M

KeyboardInterrupt: 

In [35]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

gs://jkobject/Chip_AML/Chip/mp106.fastq.gz
gs://jkobject/Chip_AML/Chip/mp109.fastq.gz
gs://jkobject/Chip_AML/Chip/mp112.fastq.gz
gs://jkobject/Chip_AML/Chip/mp115.fastq.gz
gs://jkobject/Chip_AML/Chip/mp116.fastq.gz
gs://jkobject/Chip_AML/Chip/mp117.fastq.gz
gs://jkobject/Chip_AML/Chip/mp118.fastq.gz
gs://jkobject/Chip_AML/Chip/mp122.fastq.gz
gs://jkobject/Chip_AML/Chip/mp123.fastq.gz
gs://jkobject/Chip_AML/Chip/mp124.fastq.gz
gs://jkobject/Chip_AML/Chip/mp125.fastq.gz
gs://jkobject/Chip_AML/Chip/mp127.fastq.gz
gs://jkobject/Chip_AML/Chip/mp128.fastq.gz
gs://jkobject/Chip_AML/Chip/mp129.fastq.gz
gs://jkobject/Chip_AML/Chip/mp134.fastq.gz
gs://jkobject/Chip_AML/Chip/mp136.fastq.gz
gs://jkobject/Chip_AML/Chip/mp14.fastq.gz
gs://jkobject/Chip_AML/Chip/mp143.fastq.gz
gs://jkobject/Chip_AML/Chip/mp153.fastq.gz
gs://jkobject/Chip_AML/Chip/mp154.fastq.gz
gs://jkobject/Chip_AML/Chip/mp155.fastq.gz
gs://jkobject/Chip_AML/Chip/mp156.fastq.gz
gs://jkobject/Chip_AML/Chip/mp157.fastq.gz
gs://jkobjec

In [ ]:
# EXCEPT these two... TO RETRIEVE
gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz
gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R1.fastq.gz

In [25]:
stored = !gsutil ls -a gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz
stored=stored[0]
! gsutil cp $stored gs://amlproject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz

Copying gs://jkobject/Chip_AML/Chip/mp300-MV411-RUNX1-r3_R2.fastq.gz#1573157455080605 [Content-Type=application/octet-stream]...
/ [1 files][583.3 MiB/583.3 MiB]                                                
Operation completed over 1 objects/583.3 MiB.                                    


In [ ]:
a

In [52]:
#continuing
a[85]

'gs://jkobject/Chip_AML/Chip/mp301-MV411-PU1-r2_R2.fastq.gz'

In [53]:
import re
import time
mps=[]
for i in a[86:]:
    res = re.search('mp(\d+)', i)
    if res is not None:
        print('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')
        time.sleep(2)
        os.system('gsutil mv '+i+' gs://amlproject/Chip_AML/Chip/'+res.group(0)+'.fastq.gz')

gsutil mv gs://jkobject/Chip_AML/Chip/mp302-M6-MYB-r2_S10_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp302.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp302-M6-MYB-r2_S10_R2.fastq.gz gs://jkobject/Chip_AML/Chip/mp302.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp304-D9-PU1-r2_S1_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp304.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp304-D9-PU1-r2_S1_R2.fastq.gz gs://jkobject/Chip_AML/Chip/mp304.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp305-SKNO1-PU1-r1_S2_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp305.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp305-SKNO1-PU1-r1_S2_R2.fastq.gz gs://jkobject/Chip_AML/Chip/mp305.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp308-SKNO1-RUNX1-RUNX1T1-r1_S5_R1.fastq.gz gs://jkobject/Chip_AML/Chip/mp308.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp308-SKNO1-RUNX1-RUNX1T1-r1_S5_R2.fastq.gz gs://jkobject/Chip_AML/Chip/mp308.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp319-MV411-FLI1-r1_R1.fastq.gz 

gsutil mv gs://jkobject/Chip_AML/Chip/mp715-MV411-ZEB2-r8_R1_001.fastq.gz gs://jkobject/Chip_AML/Chip/mp715.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp716-MV411-GSE1-r9_R1_001.fastq.gz gs://jkobject/Chip_AML/Chip/mp716.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp717-MV411-GATA2-r4_R1_001.fastq.gz gs://jkobject/Chip_AML/Chip/mp717.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp718-MV411-IRF2BP2-r9_R1_001.fastq.gz gs://jkobject/Chip_AML/Chip/mp718.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp72-MV411-CEBPA-r1.fastq.gz gs://jkobject/Chip_AML/Chip/mp72.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp724-MV411-DMSO-8h-MYC-r2.fastq.gz gs://jkobject/Chip_AML/Chip/mp724.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp728-MV411-DMSO-8h-CEBPA-r1.fastq.gz gs://jkobject/Chip_AML/Chip/mp728.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp73.fastq.gz gs://jkobject/Chip_AML/Chip/mp73.fastq.gz
gsutil mv gs://jkobject/Chip_AML/Chip/mp752.fastq.gz gs://jkobject/Chip_AML/Chip/mp752.fa

## adding metadata

In [5]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')

In [6]:
url="https://docs.google.com/spreadsheets/d/14onj_lD6WrjSEsE5dWJid6lGdJ81O6qpcetP5_Z0FxQ"
gsheets = sheets.get(url).sheets[1].to_frame()

In [9]:
gsheets

,ChIP No,quality,Cell line ID,Protein,Wistar BigWig URL,TF a.k.a.,Replicate No,Ab cat. no.,Ab notes,Alias,...,Number of peaks with >10 fold enrichment,Fraction of reads in peaks,Visual inspection of BigWig,QC SUMMARY,Sample a.k.a. #1,Sample a.k.a. #2,Sample a.k.a. #3,Sample a.k.a. #4,Track URL,Track Color Code
Unique sample ID,,,,,,,,,,,,,,,,,,,,,
mp1,1,NaN,D9,H3K27me3,https://shiek-db.wistar.upenn.edu/ucscTracks/p...,NaN,1,Millipore_07-449,NaN,mp1-D9-H3K27me3-r1,...,NaN,NaN,NaN,NaN,D9.H3K27me3.11,MP24,NaN,A1,track type=bigWig name=mp1-D9-H3K27me3-r1 bigD...,"204,0,0"
mp2,2,NaN,D9,H3K27ac,https://shiek-db.wistar.upenn.edu/ucscTracks/p...,NaN,1,Abcam_ab4729,NaN,mp2-D9-H3K27ac-r1,...,NaN,NaN,NaN,NaN,D9.H3K27ac.11,MP25,Project1_MP25_q42_filtered.bam.log,A2,track type=bigWig name=mp2-D9-H3K27ac-r1 bigDa...,"204,0,0"
mp3,3,NaN,D9,H3K4me1,https://shiek-db.wistar.upenn.edu/ucscTracks/p...,NaN,1,Abcam_ab8895,NaN,mp3-D9-H3K4me1-r1,...,NaN,NaN,NaN,NaN,D9.H3K4me1.11,MP26,NaN,A3,track type=bigWig name=mp3-D9-H3K4me1-r1 bigDa...,"204,0,0"
mp4,4,NaN,D9,IGG,https://shiek-db.wistar.upenn.edu/ucscTracks/p...,NaN,1,SantaCruz_sc2027,NaN,mp4-D9-IGG-r1,...,NaN,NaN,NaN,NaN,D9.IgG.11,MP29,NaN,NaN,track type=bigWig name=mp4-D9-IGG-r1 bigDataUr...,"204,0,0"
mp5,5,NaN,D9,INPUT,https://shiek-db.wistar.upenn.edu/ucscTracks/p...,NaN,1,INPUT,NaN,mp5-D9-INPUT-r1,...,NaN,NaN,NaN,NaN,D9.INPUT.11,MP30,NaN,NaN,track type=bigWig name=mp5-D9-INPUT-r1 bigData...,"204,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp774,774,NaN,MV411,HOXA9,NaN,NaN,2,Invitrogen_MA5-31766,NaN,mp774-MV411-HOXA9-r2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp775,775,NaN,P31FUJ,H3K27ac,NaN,NaN,3,Abcam ab4729,NaN,mp775-P31FUJ-H3K27ac-r3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp776,776,NaN,P31FUJ,H3K27ac,NaN,NaN,4,Abcam ab4729,NaN,mp776-P31FUJ-H3K27ac-r4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
gsheets = gsheets.set_index('Unique sample ID')

In [49]:
a = !gsutil -m ls gs://amlproject/Chip_AML/Chip/
a

['gs://jkobject/Chip_AML/Chip/mp100.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp101.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp102.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp106.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp109.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp112.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp115.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp116.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp117.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp118.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp120.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp122.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp123.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp124.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp125.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp127.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp128.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp129.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp134.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp136.fastq.gz',
 'gs://jkobject/Chip_AML/Chip/mp14.fastq.gz',
 'gs://jkobjec

In [50]:
data = pd.DataFrame({'fastq_1':a[:-1]}, index=[i.split('/')[-1].split('.')[0] for i in a[:-1]])

In [51]:
data[['quality','Protein','Replicate No','Cell line ID']] = gsheets[
    ['quality','Protein','Replicate No','Cell line ID']]

In [52]:
data['fastq_2']= np.NaN

In [53]:
data = data.rename(columns={'Protein':'antibody','Cell line ID':'group','Replicate No':'replicate'})

In [54]:
data.loc[data[data.antibody.isin(['INPUT','IGG'])].index,'group'] = \
data[data.antibody.isin(['INPUT','IGG'])].antibody +'_'+\
data[data.antibody.isin(['INPUT','IGG'])].group

In [55]:
data.loc[data[~data.antibody.isin(['INPUT','IGG'])].index,'control'] = \
'INPUT_'+\
data[~data.antibody.isin(['INPUT','IGG'])].group

In [56]:
data

,fastq_1,quality,antibody,replicate,group,fastq_2,control
mp100,gs://jkobject/Chip_AML/Chip/mp100.fastq.gz,NaN,INPUT,1,INPUT_U937,NaN,NaN
mp101,gs://jkobject/Chip_AML/Chip/mp101.fastq.gz,NaN,INPUT,1,INPUT_NOMO1,NaN,NaN
mp102,gs://jkobject/Chip_AML/Chip/mp102.fastq.gz,NaN,INPUT,1,INPUT_UT7,NaN,NaN
mp106,gs://jkobject/Chip_AML/Chip/mp106.fastq.gz,**,MYB,1,MV411,NaN,INPUT_MV411
mp109,gs://jkobject/Chip_AML/Chip/mp109.fastq.gz,NaN,CEBPA,1,M6,NaN,INPUT_M6
...,...,...,...,...,...,...,...
mp88,gs://jkobject/Chip_AML/Chip/mp88.fastq.gz,*,H3K27ac,2,MV411,NaN,INPUT_MV411
mp90,gs://jkobject/Chip_AML/Chip/mp90.fastq.gz,NaN,H3K27ac,2,U937,NaN,INPUT_U937
mp92,gs://jkobject/Chip_AML/Chip/mp92.fastq.gz,NaN,H3K27ac,2,NOMO1,NaN,INPUT_NOMO1
mp98,gs://jkobject/Chip_AML/Chip/mp98.fastq.gz,NaN,INPUT,1,INPUT_M6,NaN,NaN


In [57]:
set(data.control)-set(data.group)

{'INPUT_EOL1',
 'INPUT_F36P',
 'INPUT_HEL',
 'INPUT_MV411-DMSO',
 'INPUT_MV411-DMSO-8h',
 'INPUT_MV411-IRF2BP2-CT-GFP-8',
 'INPUT_MV411-MEF2D-NT-SC-63',
 'INPUT_MV411-PU1-NT-SC-1',
 'INPUT_MV411_DMSO_8h',
 'INPUT_SHI1',
 'INPUT_UCSDAML1',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan}

In [122]:
withoutinput = ["HEL",
                "UCSDAML1",
                "SHI1",
                "EOL1",
                "F36P"]

In [ ]:
# requesting them and downloading them...

In [24]:
rename = {'gs://amlproject/Chip_AML/Chip/1_MP5835-1of4_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp643.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_17-HL60-INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp120.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_33.U937.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp100.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_34.NOMO1.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp101.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_35.UT7.INPUT.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp102.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp309-KG1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp309.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp310-Kasumi1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp310.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp311-MOLM13-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp311.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp312-NB4-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp312.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp313-TF1-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp313.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp314-OCIAML3-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp314.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp315-MONOMAC3-INPUT-r1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp315.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/Sample_mp326-DFAM71927V3-INPUT-r1_R1.fastq.gz':"gs://amlproject/Chip_AML/Chip/mp326.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/mp609-MONOMAC1-INPUT-r1_S4.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp609.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp610-HEL9217-INPUT-r1_S5.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp610.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp611-P31FUJ-INPUT-r1_S6.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp611.fastq.gz',
 'gs://amlproject/Chip_AML/Chip/mp612-PLB985-INPUT-r1_S7.fastq.gz': 'gs://amlproject/Chip_AML/Chip/mp612.fastq.gz',
          
 'gs://amlproject/Chip_AML/Chip/1_MP5835-1of4_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/paired_end/mp643-1_MP5835-1of4_S1_R2_001.fastq.gz",          
'gs://amlproject/Chip_AML/Chip/Sample_mp326-DFAM71927V3-INPUT-r1_R2.fastq.gz':"gs://amlproject/Chip_AML/Chip/paired_end/mp326-DFAM71927V3-INPUT-r1_R2.fastq.gz"}

In [29]:
for k,v in rename.items():
    ! gsutil mv $k $v

Copying gs://jkobject/Chip_AML/Chip/mp609-MONOMAC1-INPUT-r1_S4.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://jkobject/Chip_AML/Chip/mp609-MONOMAC1-INPUT-r1_S4.fastq.gz...     

Operation completed over 1 objects/169.0 B.                                      
Copying gs://jkobject/Chip_AML/Chip/mp610-HEL9217-INPUT-r1_S5.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://jkobject/Chip_AML/Chip/mp610-HEL9217-INPUT-r1_S5.fastq.gz...      

Operation completed over 1 objects/1.1 GiB.                                      
Copying gs://jkobject/Chip_AML/Chip/mp611-P31FUJ-INPUT-r1_S6.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://jkobject/Chip_AML/Chip/mp611-P31FUJ-INPUT-r1_S6.fastq.gz...       

Operation completed over 1 objects/1.1 GiB.                                      
Copying gs://jkobject/Chip_AML/Chip/mp612-PLB985-INPUT-r1_S7.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://jkobject/Chip_AML/Chip/mp612-PLB985-INP

In [125]:
data = data.drop(data[data.group.isin(withoutinput)].index)

In [58]:
data.loc[data[~data.antibody.isin(['INPUT','IGG'])].index,'group'] = \
data[~data.antibody.isin(['INPUT','IGG'])].group + '_' + \
data[~data.antibody.isin(['INPUT','IGG'])].replicate.astype(str)

In [65]:
data[data.antibody.isin(['INPUT','IGG'])].antibody = np.nan

/home/jeremie/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [59]:
data['replicate']=1

In [60]:
data[data.columns[[4,3,0,5,2,6]]].to_csv('nextflow/design.csv', index=False)

In [3]:
! export GOOGLE_APPLICATION_CREDENTIALS=~/jkobject-b6f1adaffcb8.json
! export NXF_MODE=google
! nextflow info

  Version: 19.01.0 build 5050
  Modified: 22-01-2019 11:19 UTC 
  System: Linux 4.9.0-9-amd64
  Runtime: Groovy 2.5.5 on OpenJDK 64-Bit Server VM 1.8.0_232-8u232-b09-1~deb9u1-b09
  Encoding: UTF-8 (UTF-8)



In [4]:
! nextflow cloud create 'jkcluster' -c 40

Unknow cloud driver name: google


In [141]:
! nextflow run nf-core/chipseq -c "nextflow/nextflow.config" \
--singleEnd \
--seq_center 'DFCI' \
--email 'amlproject@gmail.com' \
--bucket-dir 'gs://amlproject/Chip_AML/nextflow/CHIPprocess_2/' \
--keyfile '~/amlproject-b6f1adaffcb8.json' \
--projectname 'amlproject' \
--zone 'us-east1-b' \
--skipDiffAnalysis \
--narrowPeak \
--design "nextflow/design.csv" \
--genome "GRCh38" \
--profile 'gcp' \
--resume \
--skipPreseq \
--max_cpus 8

N E X T F L O W  ~  version 19.01.0
Launching `nf-core/chipseq` [desperate_goldberg] - revision: 333bba3345 [master]
NOTE: Your local project version looks outdated - a different revision is available in the remote repository [21be314954]
----------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/chipseq v1.0.0
----------------------------------------------------

Run Name            : desperate_goldberg
Data Type           : Single-End
Design File         : nextflow/design.csv
Genome              : GRCh38
Fasta File          : s3://ngi-igenomes/igenomes//Homo_sapiens/NCBI/GRCh38/Sequence/WholeGenomeFasta/genome.fa
GTF File            : s3://ngi-igenomes/igenomes//Homo_sapiens/NCBI/GRCh38/Annotation/Genes/genes.gtf
BWA Index           : s3:/

In [ ]:
! nextflow cloud shutdown jkcluster

In [148]:
gsheets[gsheets['Cell line ID'].isin([
    "MOLM13",
"MONOMAC1",
"MONOMAC6",
"NB4",
"UT7",
"KG1",
"U937",
"P31FUJ",
"HL60",
"PLB985",
"OCIAML3",
"HEL9217",
"Kasumi1",
"NOMO1",
"TF1",
"SHI1",
"DFAM71927V3",
"EOL1",
"F36P"
])][gsheets.Protein=='INPUT'].index

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Index(['mp100', 'mp101', 'mp102', 'mp120', 'mp309', 'mp310', 'mp311', 'mp312',
       'mp313', 'mp314', 'mp315', 'mp326', 'mp609', 'mp610', 'mp611', 'mp612',
       'mp643'],
      dtype='object', name='Unique sample ID')

### need to use juicer
https://github.com/aidenlab/juicer/wiki/Data-Extraction

https://github.com/aidenlab/straw/wiki/Python

https://github.com/aidenlab/juicer/wiki/Data

https://github.com/aidenlab/juicer

In [39]:
! pip install hic-straw

In [40]:
import straw

In [43]:
result = straw.straw('KR', 
                     'data/GEO/MV411_H3K27ac_all_rep_duplicate_removed_allValidPairs.hic',
                     'X', 'X', 'BP', 5000)

HiC version:  8


In [47]:
len(result[1])

367172

In [37]:
! ls data/GEO

GSE71776
GSE71779
GSM3693109_MV411.rpkm.bw
MV411_H3K27ac_all_rep_duplicate_removed_allValidPairs.hic
MV411_SMC1_all_rep_duplicate_removed_allValidPairs.hic
RNAseq_perturb
slamseq
